<a href="https://colab.research.google.com/github/sowmyarshetty/NNClass/blob/main/building_cnn_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from PIL import Image
import pandas as pd
import requests
import numpy as np

# Part 1: Importing Data

In [2]:
# Reading the meta file containing all image file names
path = "https://static.bc-edx.com/ai/ail-v-1-0/m19/lesson_1/datasets/csvs/files_list.csv"

filenames_df = pd.read_csv(path)
filenames_df.head()

,files
0,ch4f_straight_angry_sunglasses.png
1,tammo_straight_neutral_sunglasses.png
2,at33_straight_neutral_open.png
3,boland_up_angry_open.png
4,mitchell_straight_angry_sunglasses.png


In [3]:
# Build a list of imported images
base_path = "https://static.bc-edx.com/ai/ail-v-1-0/m19/lesson_1/datasets/faces_data/"
images = []
for i in range(len(filenames_df)):
    filename = filenames_df.iloc[i,0]
    path = base_path + filename
    print(f'{i} of {len(filenames_df)}: Attempting to import {filename}')
    try:
        response = requests.get(path, stream=True).raw
        images.append(Image.open(response))
    except:
        print(f'FAILED: {filename}')

0 of 624: Attempting to import ch4f_straight_angry_sunglasses.png
1 of 624: Attempting to import tammo_straight_neutral_sunglasses.png
2 of 624: Attempting to import at33_straight_neutral_open.png
3 of 624: Attempting to import boland_up_angry_open.png
4 of 624: Attempting to import mitchell_straight_angry_sunglasses.png
5 of 624: Attempting to import kk49_straight_neutral_sunglasses.png
6 of 624: Attempting to import choon_left_sad_open.png
7 of 624: Attempting to import tammo_right_neutral_sunglasses.png
8 of 624: Attempting to import megak_straight_neutral_sunglasses.png
9 of 624: Attempting to import saavik_left_happy_open.png
10 of 624: Attempting to import kk49_right_neutral_sunglasses.png
11 of 624: Attempting to import mitchell_up_sad_sunglasses.png
12 of 624: Attempting to import at33_right_sad_sunglasses.png
13 of 624: Attempting to import danieln_right_sad_sunglasses.png
14 of 624: Attempting to import night_straight_happy_sunglasses.png
15 of 624: Attempting to import bolan

In [4]:
# Print a random image from the list to ensure the import was successful


## Part 2: Preprocessing

In [5]:
# Check the size of the second image
images[1].size

(32, 30)

In [6]:
# Get all the sizes into a list, then convert to a set
sizes = set([i.size for i in images])
sizes


{(32, 30), (64, 60), (128, 120)}

In [14]:
# Use a for loop to resize all images to 64 by 60
target_size = (64,60)
resized_images = []
for image in images:
  resized_images.append(image.resize(target_size))

resized_images

[<PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L size=64x60>,
 <PIL.Image.Image image mode=L s

In [36]:
# Verify the resizing of all images
# Get all the sizes into a list, then convert to a set
sizes = set(i.size for i in resized_images)
sizes

{(64, 60)}

In [37]:
# Convert all images to floating point numpy arrays
float_images = []
for image in resized_images:
  float_images.append(np.array(image).astype(np.float32))
# Display the pixel values of the first image
float_images[0]


array([[21., 19., 19., ..., 45., 45., 43.],
       [47., 43., 33., ..., 45., 45., 45.],
       [64., 66., 75., ..., 45., 45., 43.],
       ...,
       [50., 43., 40., ...,  0.,  0.,  0.],
       [49., 43., 33., ...,  0.,  0.,  0.],
       [49., 47., 33., ..., 29.,  0.,  0.]], dtype=float32)

In [38]:
# To normalize images to a range between 0 and 1,
# we need to divide all pixel values by the max of 255
normalized_images = [image/255 for image in float_images]

# Display the pixel values of the first image
normalized_images[0]

array([[0.08235294, 0.07450981, 0.07450981, ..., 0.1764706 , 0.1764706 ,
        0.16862746],
       [0.18431373, 0.16862746, 0.12941177, ..., 0.1764706 , 0.1764706 ,
        0.1764706 ],
       [0.2509804 , 0.25882354, 0.29411766, ..., 0.1764706 , 0.1764706 ,
        0.16862746],
       ...,
       [0.19607843, 0.16862746, 0.15686275, ..., 0.        , 0.        ,
        0.        ],
       [0.19215687, 0.16862746, 0.12941177, ..., 0.        , 0.        ,
        0.        ],
       [0.19215687, 0.18431373, 0.12941177, ..., 0.11372549, 0.        ,
        0.        ]], dtype=float32)

# Part 3: Labels

In [45]:
# Print the first few image filenames
filenames_df.head()


,files,userid,pose,expression,eyes,user
0,ch4f_straight_angry_sunglasses.png,ch4f,straight,angry,sunglasses,ch4f
1,tammo_straight_neutral_sunglasses.png,tammo,straight,neutral,sunglasses,tammo
2,at33_straight_neutral_open.png,at33,straight,neutral,open,at33
3,boland_up_angry_open.png,boland,up,angry,open,boland
4,mitchell_straight_angry_sunglasses.png,mitchell,straight,angry,sunglasses,mitchell


In [46]:
# First, remove the .png file extension, then split into four new columns.

filenames_df[['user', 'pose', 'expression', 'eyes']] = filenames_df['files']\
                                                            .str.replace('.png', '', regex=False)\
                                                            .str.split('_', expand=True)
filenames_df.head()



,files,userid,pose,expression,eyes,user
0,ch4f_straight_angry_sunglasses.png,ch4f,straight,angry,sunglasses,ch4f
1,tammo_straight_neutral_sunglasses.png,tammo,straight,neutral,sunglasses,tammo
2,at33_straight_neutral_open.png,at33,straight,neutral,open,at33
3,boland_up_angry_open.png,boland,up,angry,open,boland
4,mitchell_straight_angry_sunglasses.png,mitchell,straight,angry,sunglasses,mitchell


In [52]:
# Now we can call our preprocessed pixel data 'X'
X = filenames_df.drop(columns = ["user"])


# For our purposes, we'll select the userid column as 'y'
y = filenames_df["user"]


In [53]:
# Check the total number of classes
y.nunique()

20

In [54]:
# Convert values to numpy arrays
X = np.array(X)
y = np.array(y)


In [50]:
# Now we'll split our data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train.shape

(468, 5)

# Part 4: Augmentation

In [ ]:
# Apply augmentation to the whole training dataset
# Define the augmentation pipeline

# Create variables to hold the X and y training data


# Loop through all the images.

    # Select the image

    # Select the label from the training data


    # Add a channel dimension for grayscale images


    # Ensure that the input data has the correct shape


    # Add 5 images for every original image

        # Append a new image to the X list

        # Append the label for the original image to the y list


# Print the length of each list


2340
2340


In [ ]:
# Reshape test data for the model

    # Add a channel dimension for grayscale images

    # Append the image to the list


# Convert to numpy array


# Check the shape of the first image


(60, 64, 1)

# Part 5: Creating the Model

In [ ]:
# One hot encode the y data


# Convert values to numpy arrays


# Load and preprocess your CMU Face Images dataset (Ensure each image is labeled as "with sunglasses" or "without sunglasses")
# The following code assumes that you have already loaded and preprocessed your dataset into 'X' and 'y' (features and labels).

# Split the training dataset into training and validation sets


# Print the total number of one_hot_encoded columns


(1872, 20)

In [ ]:
# Define a CNN model

# Compile the model


# Train the model


Epoch 1/10
59/59 [==============================] - 14s 206ms/step - loss: 2.9054 - accuracy: 0.0940 - val_loss: 2.7589 - val_accuracy: 0.1603
Epoch 2/10
59/59 [==============================] - 11s 192ms/step - loss: 2.3450 - accuracy: 0.2110 - val_loss: 2.1557 - val_accuracy: 0.2671
Epoch 3/10
59/59 [==============================] - 10s 175ms/step - loss: 1.7476 - accuracy: 0.4279 - val_loss: 1.7969 - val_accuracy: 0.4274
Epoch 4/10
59/59 [==============================] - 12s 208ms/step - loss: 1.3308 - accuracy: 0.5534 - val_loss: 1.4197 - val_accuracy: 0.5470
Epoch 5/10
59/59 [==============================] - 11s 191ms/step - loss: 1.0221 - accuracy: 0.6651 - val_loss: 1.2233 - val_accuracy: 0.5876
Epoch 6/10
59/59 [==============================] - 11s 190ms/step - loss: 0.7765 - accuracy: 0.7388 - val_loss: 1.1127 - val_accuracy: 0.6261
Epoch 7/10
59/59 [==============================] - 11s 189ms/step - loss: 0.6170 - accuracy: 0.7895 - val_loss: 1.0302 - val_accuracy: 0.6538

In [ ]:
# Evaluate the model using the testing data


5/5 [==============================] - 0s 47ms/step - loss: 0.5037 - accuracy: 0.8846


[0.5037272572517395, 0.8846153616905212]